# About

This notebook contains tobit modeling code for linear regression. The goal is to take a look at which factors contribute to the chosen score (deprawsc) the most. Features with too many NaNs are filtered out, since the model selected cannot handle NaN values.

## Loading packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
from tobit_model4 import TobitModel
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import r2_score

and data

In [ ]:
dfOld = pd.read_csv("../data/processed/all_data_train.csv")
df_total = pd.read_csv("../data/processed/all_data_additional_train.csv")
df_val = pd.read_csv("../data/processed/all_data_validation.csv")
df = pd.concat([dfOld,df_total])

creating a constant columns necessary for tobit code

In [ ]:
df['constant'] = np.ones(len(df))
df_val['constant'] = np.ones(len(df_val))

## Max education of parents

In [ ]:
df['educ_par_max'] = df[['educ_par1', 'educ_par2']].max(axis=1)
df_val['educ_par_max'] = df_val[['educ_par1', 'educ_par2']].max(axis=1)

## Filtering columns

and filter out those that have too many NaNs (more than 10 percent)

In [ ]:
featuresPre=['constant','deprawsc', 'fincur','age','gender','race', 'educ_par_max', 'residenc', 'international', 'degree', 'gpa_sr', 'alc_any', 'exerc']

In [ ]:
features=[]
mask = ~np.isnan(df['deprawsc'])
maskval = ~np.isnan(df_val['deprawsc'])
maskp = int(sum(np.isnan(df['deprawsc']))/len(df)*100)
print(f'deprawsc has {maskp}% NaNs')
for feature in featuresPre:
    #print(feature)
    if (feature == 'deprawsc') | (feature == 'degree') | (feature == 'field'):
        continue
    markpf = int(sum(np.isnan(df[feature]))/len(df)*100)
    if (markpf > 10) & (feature != 'exerc'): 
        print('Feature ' + str(feature) + f' has too many NaNs: {markpf}%. Removing.')
    else:
        print('Feature ' + str(feature) + f' has {markpf}% NaNs. Keeping.')
        mask &= ~np.isnan(df[feature])
        maskval &= ~np.isnan(df_val[feature])
        features.append(feature)
markTotal = int(len(df[~mask])/len(df)*100)
print(f'Total {markTotal}% to be removed')

In [ ]:
features

## Intersectionality

In [ ]:
intersectionality_columns={}
intersectionality_columns_val={}
intersectionality_features=[]
#features_narrow=['gender', 'age', 'fincur', 'race', 'alc_any']
features_narrow=[ 'age', 'fincur', 'alc_any']
for feature1 in features_narrow:
    for feature2 in features_narrow:
        if feature1 == feature2:
            continue
        newfeature = feature1 + '_and_' + feature2
        oldfeature = feature2 + '_and_' + feature1
        if (not newfeature in intersectionality_features) and (not oldfeature in intersectionality_features):
            intersectionality_columns[newfeature] = df[feature1]*df[feature2]
            intersectionality_columns_val[newfeature] = df_val[feature1]*df_val[feature2]
            intersectionality_features.append(newfeature)
new_cols_df = pd.DataFrame(intersectionality_columns)
new_cols_df_val = pd.DataFrame(intersectionality_columns_val)
df = pd.concat([df, new_cols_df], axis=1)
df_val = pd.concat([df_val, new_cols_df_val], axis=1)
features += intersectionality_features

In [ ]:
intersectionality_features

## Train-Test split

In [ ]:
mask.value_counts()

In [ ]:
df_train, df_test = train_test_split(df[mask],shuffle=True,random_state=440,test_size=0.2)

In [ ]:
Xfiltered=df_train[features]
Yfiltered=df_train['deprawsc'].values

In [ ]:
Xtest=df_test[features]
Ytest=df_test['deprawsc'].values

In [ ]:
Xval=df_val[maskval][features]
Yval=df_val[maskval]['deprawsc'].values

## Pipeline

In [ ]:
categorical_features=['educ_par_max','gender','race','international','residenc','alc_any']
numerical_features=['gpa_sr','fincur','age','exerc'] + intersectionality_features
const_features=['constant']
numerical_transformer=StandardScaler()
const_transformer='passthrough'
categorical_transformer=OneHotEncoder(handle_unknown='ignore', drop=[1,2,7,0,1,0])
preprocessor = ColumnTransformer(
    transformers=[
        ('con', const_transformer, const_features),
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

## Modeling

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', TobitModel(ul=27,lol=0))
])

In [ ]:
pipeline.fit(Xfiltered, Yfiltered)

In [ ]:
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
coefficients = pd.Series(pipeline.named_steps['model'].params_[:-1], index=feature_names)
pvalues = pd.Series(np.round(pipeline.named_steps['model'].p_values[:-1],3), index=feature_names)

## Looking at results

In [ ]:
impact_dict={feature: coefficients[feature] for feature in feature_names}

In [ ]:
sorted_impact = sorted(impact_dict.items(), key=lambda item: np.abs(item[1]),reverse=True)
sorted_impact_asc = list(dict(sorted_impact))#[:20]
print('Top features: ')
for feature in sorted_impact_asc:
    print(str(feature) + ' has impact ' + str(dict(sorted_impact)[feature]) + ' w/ p value ' + str(pvalues[feature]))

In [ ]:
# remove those with low p value
sorted_impact_all = list(dict(sorted_impact))
significant_features=[]
for feature in sorted_impact_all:
    if pvalues[feature] < 0.05:
        significant_features.append(feature)
print('Top (significant) features: ')
for feature in significant_features:
    print(str(feature) + ' has impact ' + str(dict(sorted_impact)[feature]) + ' w/ p value ' + str(pvalues[feature]))

## Confusion matrix etc.

In [ ]:
threshold = np.median(Ytest)
print(threshold)
thresholdVal = np.median(Yval)
print(thresholdVal)

In [ ]:
preds = pipeline.predict(Xtest)
preds[preds > 27] = 27
preds[preds < 0] = 0


preds_happy = (preds < threshold).astype(int)
y_val_happy = (Ytest < threshold).astype(int)

cm = confusion_matrix(y_val_happy, preds_happy)

TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
TP = cm[1, 1]

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * (precision * recall) / (precision + recall)

# Define display labels if desired
display_labels = ['unhappy', 'happy']

# Create a display object
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)

# Plot the matrix
cm_display.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

print('accuracy = ', str(accuracy*100))
print('precision = ', str(precision*100))
print('recall = ', str(recall*100))
print('f1 = ', str(f1*100))

In [ ]:
predsval = pipeline.predict(Xval)
predsval[predsval > 27] = 27
predsval[predsval < 0] = 0
tobis_predict=pipeline.predict(Xval)
tobis_mse=mse(Yval,tobis_predict)


valpreds_happy = (predsval < thresholdVal).astype(int)
valy_val_happy = (Yval < thresholdVal).astype(int)

cm = confusion_matrix(valy_val_happy, valpreds_happy)

TNV = cm[0, 0]
FPV = cm[0, 1]
FNV = cm[1, 0]
TPV = cm[1, 1]

accuracyV = (TPV + TNV) / (TPV + TNV + FPV + FNV)
precisionV = TPV / (TPV + FPV)
recallV = TPV / (TPV + FNV)
f1V = 2 * (precisionV * recallV) / (precisionV + recallV)

# Define display labels if desired
display_labels = ['unhappy', 'happy']

# Create a display object
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)

# Plot the matrix
cm_display.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

print('accuracy = ', str(accuracyV*100))
print('precision = ', str(precisionV*100))
print('recall = ', str(recallV*100))
print('f1 = ', str(f1V*100))
print(f'MSE = {tobis_mse}')